In [1]:
# keras module for building LSTM
import json
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

# set seeds for reproducability
import tensorflow as tf

import pandas as pd
import numpy as np
import string, os

import warnings
import matplotlib.pyplot as plt
from keras.preprocessing.text import tokenizer_from_json
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

from dataset import load_tokenized_sentences

LEN_MIN_LIMIT = 5
LEN_MAX_LIMIT = 45
SKIP = 3

dataset = load_tokenized_sentences('../datasets/words/books-bajki-raw.pickle')

with open('../datasets/words/books-bajki-raw-tokenizer_100000.json', 'r') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
        
print({
    "size": len(dataset),
    "tokenizer": tokenizer.num_words,
    "str": dataset[1][:500],
    "tokenizer": tokenizer.num_words
})

{'size': 144, 'tokenizer': 100000, 'str': 'Louisa May Alcott\n\nMałe kobietki\ntłum. Zofia Grabowska\n\nISBN 978-83-288-6176-3\n\n\n\n\nRozdział I. Pielgrzymki\n\n— Co mi za Boże Narodzenie bez podarków! — mruknęła Ludka, leżąc na dywanie przed kominkiem.\n\n— To straszne być ubogą! — westchnęła Małgosia, spoglądając na swą starą suknię.\n\n— Bardzo jest nieładnie, że niektóre dziewczęta mają mnóstwo pięknych rzeczy, a inne nie mają nic — dodała Amelka z gniewną minką.\n\n— Mamy przecież ojca, mamę i siebie nawzajem — z zadowoleniem odezwała się Eliza ze '}


In [2]:
from typing import List


def get_sequence_of_tokens(dataset: List[str], tokenizer: Tokenizer):
    word2int_sequences = []
    for seq_text in tokenizer.texts_to_sequences(dataset):
        for sindex in range(0, len(seq_text) - LEN_MAX_LIMIT, SKIP):
            rlen = np.random.randint(LEN_MIN_LIMIT, LEN_MAX_LIMIT)
            n_gram_sequence = seq_text[sindex:sindex + rlen]
            word2int_sequences.append(n_gram_sequence)
    return word2int_sequences


def generate_padded_sequences(word2int_sequences: List[List[int]]):
    word2int_sequences = np.array(
        pad_sequences(word2int_sequences, maxlen=LEN_MAX_LIMIT, padding="pre")
    )
    predictors, label = word2int_sequences[:, :-1], word2int_sequences[:, -1]
    return predictors, label


def dataset_generator(dataset: List[str], tokenizer: Tokenizer, batch_size=256):
    flag = True
    while True:
        word2int_sequences = get_sequence_of_tokens(dataset, tokenizer)
        predictors, label = generate_padded_sequences(word2int_sequences)

        if flag:
            flag = False
            print('word2int_sequences', len(word2int_sequences))
        
        p = np.random.permutation(len(predictors))
        for i in range(0, len(predictors) - batch_size + 1, batch_size):
            indexes = p[i : i + batch_size]
            yield predictors[indexes], label[indexes]

total_words = tokenizer.num_words + 1
batch_size = 256
gen = dataset_generator(dataset, tokenizer, 256)
a, b = next(gen)
print(a.shape, b.shape)

word2int_sequences 2238020
(256, 44) (256,)


In [3]:
import random


class PredictCallback(tf.keras.callbacks.Callback):
    def __init__(self, seed_text, next_words, max_sequence_len, temperature= 0.0):
        self.seed_text = seed_text
        self.next_words = next_words
        self.max_sequence_len = max_sequence_len
        self.temperature = temperature

    def sample(self, preds: np.ndarray):
        if self.temperature > 0:
            preds = np.asarray(preds).astype("float64")
            preds = np.log(preds) / self.temperature
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            preds = np.random.multinomial(1, preds, 1)
        return np.argmax(preds)
    
    def on_epoch_begin(self, epoch, logs=None):
        seed_text: str = self.seed_text
        for _ in range(self.next_words):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences(
                [token_list], maxlen=self.max_sequence_len - 1, padding="pre"
            )
            predicted = self.model.predict_on_batch(token_list)[0]
            predicted = self.sample(predicted)
            output_word = tokenizer.index_word[predicted]
            seed_text += " " + output_word
        print(f"Start epoch {epoch} of training; Temperature: {self.temperature:.1f} Generated text:", seed_text)


def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(total_words, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
    return model

print(total_words, LEN_MAX_LIMIT)
model = create_model(LEN_MAX_LIMIT, total_words)
model.summary()

"""
1. dodanie do zbioru znaków .,?!;:
2. dodanie temperatury
3. usunięcie niektórych słów 
4. dodanie bidirectional
5. ograniczenie wyjścia dense 
6. przygotowanie zbioru
"""


100001 45
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 44, 50)            5000050   
                                                                 
 bidirectional (Bidirectiona  (None, 44, 512)          628736    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 44, 1024)         4198400   
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 44, 1024)          0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             6295552   
 nal)                                                            
                                              

'\n1. dodanie do zbioru znaków .,?!;:\n2. dodanie temperatury\n3. usunięcie niektórych słów \n4. dodanie bidirectional\n5. ograniczenie wyjścia dense \n6. przygotowanie zbioru\n'

In [4]:
model.fit(
    gen,
    steps_per_epoch=2238506 // batch_size,
    epochs=20,
    verbose=1,
    callbacks=[
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0.1),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0.2),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 0.5),
        PredictCallback("dawno temu czerwony kapturek poszedł do lasu", 35, LEN_MAX_LIMIT, 1.0),
        tf.keras.callbacks.ModelCheckpoint('../lstm_models/model_best_3.h5', monitor='loss', save_best_only=True, save_weights_only=False)
    ],
)

Start epoch 0 of training; Temperature: 0.0 Generated text: dawno temu czerwony kapturek poszedł do lasu głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, głupie, ułożoną głupie, głupie, ułożoną głupie, głupie, ułożoną głupie, głupie, ułożoną ułożoną głupie, sto­pą sto­pą sto­pą ułożoną martwcie martwcie hiszpan,
Start epoch 0 of training; Temperature: 0.1 Generated text: dawno temu czerwony kapturek poszedł do lasu zakończonym odrażające tematu wyspać znosi, gła­dząc włóczęgi, usadowił poszukiwaniach gromki uznawać biega otaczając potargane doczekać, nasłuchując, brudną po­cią­gnął wyrywać błogości najlepszy, odstąpi pomieszaniu schodzi maurów, kupuje pozoru, miesiąc, pojedynek. zaj­dzie ona... wyleczysz wie­dzą, odmiennej kapeli
Start epoch 0 of training; Temperature: 0.2 Generated text: dawno temu czerwony kapturek poszedł do lasu miastu mó­wię, al­bert spróbujmy modlitwę. wiadra krzyknąć ścieżki, ucie­chę py­ta­n